<div>
  <h1>
    <img src="https://langchain-ai.github.io/langgraph/static/wordmark_dark.svg" style="height: 40px; vertical-align: bottom;" />
    +
    <img src="https://kumo-ai.github.io/kumo-sdk/docs/_static/kumo-logo.svg" style="height: 40px; vertical-align: bottom;""/>
    RFM Insurance agent demo
  </h1>
</div>

This notebook demonstrates how to combine **LangGraph** with the **`KumoRFM` MCP server** to implement a insurance policy risk and retention workflow.  

The workflow includes:

1. **Data Ingestion & Graph Materialization**  
   Load S3 Parquet files into a `KumoRFM` graph, validate schemas, and summarize the resulting graph.

2. **Policy expiration prediction**  
   Compute churn probabilities for the selected users using predictive queries via `KumoRFM`, highlighting high-risk policies.

3. **Policy bundling recommendation**  
   Predict which policies at-risk users are most likely interested in, leveraging predictive queries and table lookups.

4. **Re-Engagement Email Drafting**  
   Automatically draft personalized, persuasive emails including recommended policy bundles and optional discount codes to encourage policy retention and upsell.

This demo illustrates a simple **ReAct agent orchestration** for data engineering, analytics, and policy retention/upsell using LangGraph and MCP tools.

## Running this notebook
1. Create a new environment with `uv` or `pip`
2. Install required packages:
```
# Install required libraries
pip install langchain-core>=0.1.0
pip install langgraph>=0.0.40
pip install langchain-mcp-adapters>=0.1.0
pip install langchain-anthropic>=0.1.0
pip install kumo-rfm-mcp>=0.1.0
pip install s3fs>=2025.09.0
pip install fsspec>=2025.09.0
pip install jupyter
```
3. (Optional) Obtain the Kumo API key at [kumorfm.ai](https://kumorfm.ai/api-keys) and set it as an environment variable `export KUMO_API_KEY=<your-api-key>`
4. Run the notebook via `jupyter notebook`

In [1]:
# Install required libraries
!pip install langchain-core>=0.1.0
!pip install langgraph>=0.0.40
!pip install langchain-mcp-adapters>=0.1.0
!pip install langchain-anthropic>=0.1.0
!pip install kumo-rfm-mcp>=0.1.0
!pip install s3fs>=2025.09.0
!pip install fsspec>=2025.09.0

zsh:1: 0.1.0 not found
zsh:1: 0.0.40 not found
zsh:1: 0.1.0 not found
zsh:1: 0.1.0 not found
zsh:1: 0.1.0 not found
zsh:1: 2025.09.0 not found
zsh:1: 2025.09.0 not found


In [2]:
# Import required libraries
import asyncio
import os
import json
from typing import TypedDict, Annotated, List
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt import create_react_agent, ToolNode, tools_condition
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools  # Add this import
from langchain_anthropic import ChatAnthropic

import pandas as pd
from datetime import datetime, timedelta

import kumoai.experimental.rfm as rfm

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## Set API Keys
To run this example notebook you will need both a KumoRFM and an Anthropic API key. You can generate the keys by visiting [kumorfm.ai](https://kumorfm.ai/api-keys) and [console.anthropic.com](https://console.anthropic.com/settings/keys) respectively. Make sure to update the values in the cell below! 

In [ ]:
# API KEYS
if not os.environ.get("KUMO_API_KEY"):
    rfm.authenticate()

# or for manual setup:
# os.environ['KUMO_API_KEY'] = "<KUMO_API_KEY>"

import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

## Setup KumoRFM MCP config

LangGraph itself doesn't provide out-of-the-box support for MCP servers, instead it uses it's own tool definitions - see [documentation](https://python.langchain.com/docs/concepts/tools/). But the LangGraph team does provide a helpful toolset to transform MCPs into LangGraph tool calls `langchain-mcp-adapters` ([github](https://github.com/langchain-ai/langchain-mcp-adapters)).

We can take a MCP server configuration and pass it to the `MultiServerMCPClient` class which we can use as a context manager to obtain all the tool calls we need from our server, and it is really as simple as that!

```{python}
from langchain_mcp_adapters.tools import load_mcp_tools

client = MultiServerMCPClient(mcp_servers)
async with client.session("math") as session:
    tools = await load_mcp_tools(session)
```

We define the config for `kumo-rfm-mcp` below.

In [6]:
# MCP Server Configuration
mcp_servers = {
    "kumo-rfm": {
        "command": "python3",
        "args": ["-m", "kumo_rfm_mcp.server"],
        "env": {
            "KUMO_API_KEY": os.getenv("KUMO_API_KEY"),
        },
        "transport": "stdio",
    },
}

## Data config and agent prompt
Now to the subject of data, we will be using a multi-table dataset which includes insurance customers, policies, claims, agents, and more. We have made the dataset publicly available at: `s3://kumo-sdk-public/rfm-datasets/insurance/`. We provide this as part of the prompt to the agent. 

Let us first identify the customers who have expiring policies in the next 30 days:

In [16]:
DATA_DIR = "s3://kumo-sdk-public/rfm-datasets/insurance"
TEST_DATE = datetime(day=3, month=9, year=2025)

In [18]:
policies_df = pd.read_csv(f"{DATA_DIR}/policies.csv")
policies_df['expiration_date'] = pd.to_datetime(policies_df['expiration_date'])

DAYS = 30 # Day limit till expiration
days_from_now = TEST_DATE + timedelta(days=30)

# Filter for policies expiring within the next 30 days, not canceled, and status not 'Canceled'
expiring_policies_df = policies_df[
    (policies_df['expiration_date'].notna()) &
    (policies_df['expiration_date'].dt.date >= datetime.now().date()) &
    (policies_df['expiration_date'].dt.date <= days_from_now.date()) &
    ((policies_df['canceled_flag'].isna()) | (policies_df['canceled_flag'] == False)) &
    (policies_df['status'] != 'Canceled')
]

CUSTOMER_IDs = expiring_policies_df['customer_id'].unique().tolist()

print(f"Customer IDs with policies expiring within the next 30 days of {TEST_DATE}:")
print(CUSTOMER_IDs)

Customer IDs with policies expiring within the next 30 days of 2025-09-03 00:00:00:
[13, 402, 14, 329, 375, 239, 493, 319]


You can find the agent prompt below, feel free to modify it to get a feel for how the models respond to different instructions. In its current form it executes a complex workflow, including inspecting the tables, defining the graph, running several predictions and finally offering up customized emails with insurance bundling options.

In [19]:
# Environment Variables
PROMPT = f"""You are an Insurance Churn Prevention Agent. You will be provided a data path and list of file names and a set of customer IDs with policies expiring in the next 30 days. Perform this multi-step workflow:

## STEP 1: Setup and Authentication
1. Start by authenticating with the KumoRFM system using `kumo-rfm:authenticate` tool
2. Find all table files in the data directory using `kumo-rfm:find_table_files` 
3. Inspect all tables using `kumo-rfm:inspect_table_files` to understand the data structure

## STEP 2: Graph Design and Setup
4. Use your domain knowledge to determine optimal graph structure:
   - Identify primary keys and time columns for each table
   - Determine appropriate semantic types for each column
   - Map foreign key relationships between tables
5. Explain your chosen graph structure and reasoning
6. Initialize the graph using `kumo-rfm:update_graph_metadata` to:
   - Add all tables with correct primary keys and time columns
   - Set appropriate semantic types for each column
   - Create foreign key-primary key links between tables
7. Materialize the graph using `kumo-rfm:materialize_graph`

## STEP 3: Renewal Prediction Analysis
8. Predict likelihood of each customer renewing their policy in next 60 days using PQL query:
   ```
   PREDICT COUNT(policies.*, 0, 60, days) > 0 
   FOR customers.customer_id IN (customer_list) 
   WHERE COUNT(policies.*, -365, 0, days) > 0
   ```
9. Categorize customers by churn risk:
   - HIGH RISK: < 30% renewal probability
   - MEDIUM RISK: 30-70% renewal probability  
   - LOW RISK: > 70% renewal probability
10. Summarize renewal prediction results and provide strategic recommendations

## STEP 4: Cross-Selling Analysis
11. For ALL customers (not just those likely to renew), predict which insurance products they're most interested in using:
    ```
    PREDICT LIST_DISTINCT(policies.product_id, 0, 60, days) RANK TOP 5 
    FOR customers.customer_id IN (customer_list)
    ```
12. Look up product names using `kumo-rfm:lookup_table_rows` on the products table
13. Analyze cross-selling opportunities and identify top recommended products per customer

## STEP 5: Personalized Email Generation
14. Generate personalized retention emails for each customer that include:
    - Professional, concise tone (no emojis or overly casual language)
    - Acknowledgment of their expiring policy
    - Renewal discount offer (15-25% off current policy)
    - Bundle discount offer for their current + most recommended new product
    - Clear call-to-action with contact information
    - Sense of urgency but not pushy
15. Customize each email based on:
    - Customer's churn risk level (adjust discount percentage)
    - Their top recommended product for bundling
    - Professional insurance industry language

## STEP 6: Strategic Summary
16. Provide executive summary with:
    - Overall churn risk assessment
    - Revenue impact analysis
    - Recommended immediate actions
    - Success metrics to track

## Data Parameters:
Data Path: {DATA_DIR}
File Names: agents.csv, claim_reports.csv, claims.csv, customers.csv, pets.csv, policies.csv, products.csv, properties.csv, vehicles.csv
Customer IDs: {CUSTOMER_IDs}

## Expected Insurance Domain Knowledge:
- Policies table connects customers to their insurance policies
- Claims table tracks insurance claims made against policies
- Products represent different insurance types (auto, home, renters, etc.)
- Agents are insurance sales representatives
- Properties, vehicles, pets are assets covered by specific policy types
- Time columns are critical for preventing data leakage in predictions
- Foreign keys typically follow pattern: policy_id links to policies, customer_id links to customers, etc.

## Key Success Criteria:
- Graph must be properly structured with correct relationships
- Predictions must use appropriate temporal windows
- Emails must be professional and personalized
- Analysis must provide actionable business insights
- All steps must be executed in sequence with proper error handling
"""

## Setting up the main agentic loop
The `async def main()` function orchestrates an AI insurance agent that combines LangGraph for agent management with MCP (Model Context Protocol) for tool integration. It creates a ReAct agent powered by Claude 3.5 Sonnet that can use real external tools through a persistent MCP session.

### Key workflow:
The setup initializes Claude as the language model, establishes a persistent connection to MCP servers via `MultiServerMCPClient` client, and dynamically loads available tools. The persistent session is crucial - it maintains state across tool calls so the agent can build on previous work rather than starting fresh each time.

### LangGraph streaming and parsing:
Instead of waiting for a final result, `agent.astream()` provides real-time updates as the agent works. LangGraph streams back chunks containing different node types:
* `"agent"` nodes: The AI's reasoning and planning process
* `"tools"` nodes: Actual tool executions with both the tool name and results

This streaming approach lets you watch the agent's decision-making in real-time. The `recursion_limit: 50` prevents infinite loops while allowing complex multi-step problem solving.

The async context manager ensures clean resource management, automatically closing the MCP session when complete. Error handling provides detailed feedback if anything goes wrong during execution.

In [20]:
async def main():
    """Main execution function using persistent MCP session."""
    
    print("🚀 Starting Insurance Agent Workflow with LangGraph + MCP")
    print("=" * 60)
    
    try:
        # Initialize the chat model
        model = ChatAnthropic(
            model="claude-3-7-sonnet-20250219",
            api_key=os.getenv("ANTHROPIC_API_KEY")
        )
        
        # Setup MCP client with persistent session
        client = MultiServerMCPClient(mcp_servers)
        
        print("🔌 Establishing persistent MCP session...")
        
        # Use persistent session to maintain state across tool calls
        async with client.session("kumo-rfm") as session:
            print("✅ MCP session established")
            
            # Load tools from the persistent session  
            tools = await load_mcp_tools(session)
            print(f"🛠️  Loaded {len(tools)} tools from MCP server")
            
            # Create the ReAct agent with MCP tools
            agent = create_react_agent(
                model, 
                tools,
            )
            
            # Execute the agent with streaming
            print("\n🤖 Agent is starting work...")
            print("=" * 40)
            
            # Stream the agent's work in real-time
            async for chunk in agent.astream(
                {"messages": [HumanMessage(content=PROMPT)]},
                {"recursion_limit": 50},
            ):

                # Parse chunk content based on type
                for node, content in chunk.items():
                    if node == "agent":
                        print("🤖 Agent:")
                        for message in content["messages"]:
                            if hasattr(message, 'content'):
                                print(f"   AI: {message.content}")
                    elif node == "tools":
                        print("🛠️  Tools:")
                        for message in content["messages"]:
                            if hasattr(message, 'content'):
                                print(f"   Tool Result: {message.content}")
                            if hasattr(message, 'name'):
                                print(f"   Tool Name: {message.name}")
                print("-" * 40)
        
        print("\n✅ Workflow completed! MCP session closed.")
                
    except Exception as e:
        print(f"❌ Agent execution failed: {e}")
        import traceback
        traceback.print_exc()

## Running the agent
We can run the main function in a IPython notebook with a simple `await` - run and see for yourself! 

In [21]:
await main()

🚀 Starting Insurance Agent Workflow with LangGraph + MCP
🔌 Establishing persistent MCP session...
✅ MCP session established
🛠️  Loaded 11 tools from MCP server

🤖 Agent is starting work...
🤖 Agent:
   AI: [{'text': "# Insurance Churn Prevention Analysis\n\nI'll help you execute the multi-step workflow to analyze and prevent customer churn for the insurance company. Let's proceed step-by-step:\n\n## STEP 1: Setup and Authentication\n\nLet me start by authenticating with the KumoRFM system:", 'type': 'text'}, {'id': 'toolu_01S48Kr6AgPHDq2rbACtx2s6', 'input': {}, 'name': 'authenticate', 'type': 'tool_use'}]
----------------------------------------
🛠️  Tools:
   Tool Result: KumoRFM session successfully authenticated
   Tool Name: authenticate
----------------------------------------
🤖 Agent:
   AI: [{'text': "Now, let's find all table files in the specified S3 directory:", 'type': 'text'}, {'id': 'toolu_01QCgdMyERuPESCGNAFTjm3g', 'input': {'path': 's3://kumo-sdk-public/rfm-datasets/insura

## We'd love to hear from you! ❤️

1. **Found a bug or have a feature request?**  
   Submit issues directly on [GitHub](https://github.com/kumo-ai/kumo-rfm). Your feedback helps us improve RFM for everyone.

2. **Built something cool with RFM? We'd love to see it!**  
   Share your project on LinkedIn and tag @kumo. We regularly spotlight on our official channels—yours could be next!

<div align="left">
  <img src="https://kumo-sdk-public.s3.us-west-2.amazonaws.com/rfm-colabs/kumo_ai_logo.jpeg" width="30" />
</div>